<a href="https://colab.research.google.com/github/Reennon/multigec-models/blob/main/notebooks/aya_expanse_8b/multigec/omnigec_minimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from google.colab import userdata

os.environ["GIT_TOKEN"] = userdata.get('git_token')

In [2]:
!git clone https://$GIT_TOKEN@github.com/Reennon/multigec-models.git

Cloning into 'multigec-models'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 157 (delta 54), reused 65 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (157/157), 181.32 KiB | 4.77 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [3]:
%cd multigec-models

/content/multigec-models


In [4]:
!git pull

Already up to date.


In [6]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit -q
!pip install flash-attn --no-build-isolation -q
!pip install wtpsplit==2.1.1 -q
!pip install syntok==1.4.4 -q
!pip install omegaconf -q
!pip install wandb -q
!pip install --upgrade transformers trl -q
!pip install pandas numpy -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [7]:
import os

from omegaconf import OmegaConf
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from src.utils.multigec import sentences, LANG_TO_CODE, LANG_CODE_TO_TOKEN
from langchain_core.prompts import PromptTemplate

from src.instruction_templates import multigec_prompts

import torch
import wandb

from transformers import BitsAndBytesConfig
from tqdm import tqdm
from trl.trainer import ConstantLengthDataset
import pandas as pd
from datasets import Dataset
from transformers.trainer_callback import EarlyStoppingCallback

from transformers import TrainingArguments
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig

tqdm.pandas()

In [8]:
parameters = OmegaConf.load("./params/aya_expanse_8b.yaml")

In [9]:
track     = "minimal"
model_name = "aya-expanse-8b"
fine_tuned_model_name = f"aya-expanse-8b-omnigec-{track}"
experiment_name = f"multigec-{track}-{model_name}"

hf_key   = userdata.get("hf_key")
secret_wandb = userdata.get("wandb_key")
in_path  = f"/gdrive/MyDrive/multigec/datasets/omnigec_{track}.csv"

# Path where the output will be saved to
out_path = f"/gdrive/MyDrive/multigec/preds/multigec_test_{track}.csv"
out_model_dir = f"/gdrive/MyDrive/multigec/multigec/models/{fine_tuned_model_name}"
QUANTIZE_4BIT = True
device   = "cuda:0"

In [10]:
wandb_project_name = f'{model_name.upper()}-omnigec-{track}'

wandb.login(key = secret_wandb)

/usr/local/lib/python3.11/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rkovalch (rkovalchuk) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
login(hf_key)

In [12]:
import gc

def clear_gpu_memory():
    torch.cuda.empty_cache()
    print(gc.collect())
clear_gpu_memory()

220


In [13]:
!env TORCH_USE_CUDA_DSA=1

SHELL=/bin/bash
NV_LIBCUBLAS_VERSION=12.5.3.2-1
NVIDIA_VISIBLE_DEVICES=all
COLAB_JUPYTER_TRANSPORT=ipc
NV_NVML_DEV_VERSION=12.5.82-1
NV_CUDNN_PACKAGE_NAME=libcudnn9-cuda-12
CGROUP_MEMORY_EVENTS=/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events
NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.22.3-1+cuda12.5
NV_LIBNCCL_DEV_PACKAGE_VERSION=2.22.3-1
VM_GCE_METADATA_HOST=169.254.169.253
HOSTNAME=b78b45ea3c06
LANGUAGE=en_US
TBE_RUNTIME_ADDR=172.28.0.1:8011
COLAB_TPU_1VM=
GCE_METADATA_TIMEOUT=3
NVIDIA_REQUIRE_CUDA=cuda>=12.5 brand=unknown,driver>=470,driver<471 brand=grid,driver>=470,driver<471 brand=tesla,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=vapps,driver>=470,driver<471 brand=vpc,driver>=470,driver<471 brand=vcs,driver>=470,driver<471 brand=vws,driver>=470,driver<471 brand=cloudgaming,driver>=470,driver<471 brand=unknown,driver>=535,dr

In [14]:
checkpoint = "CohereForAI/aya-expanse-8b"
quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
config = AutoConfig.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    config=config,
    quantization_config=quantization_config,
    torch_dtype="bfloat16",
    device_map=device,
    attn_implementation="flash_attention_2",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/8.64k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [15]:
multigec_df = pd.read_csv(in_path)
train_df = multigec_df.loc[multigec_df.loc[:, "split"] == "train"]
val_df = multigec_df.loc[multigec_df.loc[:, "split"] == "val"]

In [16]:
def formatting_prompts_func(example):
    language_code = LANG_TO_CODE[example["language"]]
    language_token = LANG_CODE_TO_TOKEN[language_code]

    user_input = example['feature']
    prompt_template = multigec_prompts[example["language"]].prompt_template
    instruction = prompt_template.format(original_text=user_input)
    target = example['target']

    text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{target}"

    return text

In [17]:
num_added_toks = tokenizer.add_tokens(
    [v for v in LANG_CODE_TO_TOKEN.values()],
    special_tokens=True
)
model.resize_token_embeddings(len(tokenizer))

Embedding(255040, 4096, padding_idx=0)

In [18]:
training_dataset = Dataset.from_pandas(train_df).shuffle()
val_dataset = Dataset.from_pandas(val_df).shuffle()

seq_length = 1600

cld_train_dataset = ConstantLengthDataset(
    tokenizer=tokenizer,
    dataset=training_dataset,
    seq_length=seq_length,
    eos_token_id=tokenizer.eos_token_id,
    shuffle=True,
    append_concat_token=True,
    add_special_tokens=True,
    formatting_func=formatting_prompts_func,
)
cld_val_dataset = ConstantLengthDataset(
    tokenizer=tokenizer,
    dataset=val_dataset,
    seq_length=int(seq_length/2),
    eos_token_id=tokenizer.eos_token_id,
    shuffle=True,
    append_concat_token=True,
    add_special_tokens=True,
    formatting_func=formatting_prompts_func,
)

In [19]:
# parameters.training["gradient_accumulation_steps"] = 8
parameters.training["per_device_train_batch_size"] = 5
# parameters.training["per_device_eval_batch_size"] = 2
# # Try these
# parameters.training["lr_scheduler_type"] = "cosine"
# parameters.training["max_grad_norm"] = 1.0
# parameters.training["output_dir"] = "results"
parameters.training["num_train_epochs"] = 12 # be sure to set 8 for fluency
# parameters.training["eval_strategy"] = "steps"
# parameters.training["save_strategy"] = "steps"        # Save model after every step
# parameters.training["metric_for_best_model"] = "eval_loss"
# parameters.training["greater_is_better"] = False
# parameters.training["group_by_length"] = False # we already use CLD
# parameters.training["save_total_limit"] = 3           # Keep only the 3 most recent checkpoints
# parameters.training["load_best_model_at_end"] = True # EarlyStoppingCallback requires load_best_model_at_end = True
# parameters.training["warmup_steps"] = 50
# parameters.training["learning_rate"] = 3e-5
parameters.training["save_steps"] = 100
parameters.training["eval_steps"] = 100
# parameters.training["weight_decay"] = 0.0
# early_stopping_patience = 75

In [ ]:
run = wandb.init(
    project=wandb_project_name,
    job_type="training",
    anonymous="allow"
)

wandb.config.update(dict(parameters.training))

peft_config = LoraConfig(
    r=parameters.lora.r,
    lora_alpha=parameters.lora.lora_alpha,
    target_modules=list(parameters.lora.target_modules),
    bias=parameters.lora.bias,
    task_type=parameters.lora.task_type
)
training_arguments = SFTConfig(
    **parameters.training,
    packing=True,
    max_seq_length=seq_length,
    output_dir=out_model_dir,
    ignore_data_skip=True,
)
trainer = SFTTrainer(
    model=model,
    train_dataset=cld_train_dataset,
    eval_dataset=cld_val_dataset,
    peft_config=peft_config,
    args=training_arguments,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=parameters.early_stopping.early_stopping_patience)],
    #formatting_func=formatting_prompts_func,
)

with torch.backends.cuda.sdp_kernel(
    enable_flash=True,
    enable_math=False,
    enable_mem_efficient=False
):
    trainer.train(
        resume_from_checkpoint=True,
    )

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_config.py:184: DeprecationWarning: `max_seq_length` is deprecated and will be remove in version 0.20.0. Use `max_length` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_config.py:184: DeprecationWarning: `max_seq_length` is deprecated and will be remove in version 0.20.0. Use `max_length` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides ba

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


In [ ]:
from google.colab import runtime
runtime.unassign()